# Python interface for Cy-RSoXS

 - Copyright @ Iowa State University
 - Distributed freely under MIT license.
 - Current Version = 0.8.2
 - Comments/Questions :    
        1. Dr. Baskar GanapathySubramanian (baskarg@iastate.edu)     
        2. Dr. Adrash Krishnamurthy        (adarsh@iastate.edu)                                 
 

## Notebook Dependencies

The notebook has following dependencies:

- python3 (Version >= 3.6)
- numpy
- pandas
- h5py (For HDF related utilities)


# Running the notebook

In order to run this notebook, you need to copy the following files in your current directory from the Cy-RSoXS directory:
* OpticalConstants/PEOlig2018.txt
* Data/edgespheres32.hd5

## Interface Overview:

The following input are required to run the Cy-RSoXS:

- Input Data parameters.
- Optical constants data at different energies calculate the refractive index.
- The morphology data.

### Preprocessing Step 0: Import the path to the library.
You should have `CyRSoXS.so` located in the directory

In [ ]:
import sys
sys.path.append("/home/maksbh/Documents/work/cy-rsoxs/cmake-build-debug") 

In [ ]:
# import the relevant modules
import h5py
import CyRSoXS as cy
import pandas as pd
import numpy as np

## Preprocessing Step 2: Computing Optical constants from file

* This examples shows a way to read the optical constants from a given file. But it can be done according to the user demands. The final goal at the end of this step is to return the optical constants for a given Energy as a `list` 

In [ ]:
# label for the column for the respective energies.
# Note: The column starts from 0    

labelEnergy={"BetaPara":0,
                 "BetaPerp":1,
                 "DeltaPara":2,
                 "DeltaPerp":3,
                 "Energy":6}

In [ ]:
def generateDataFrame(filename,labelEnergy,sep='\s+'):
    '''
    Returns DataFrame of the Energy
     
         Parameters:
             filename    (String) : The path where the filename is located.
             labelEnergy (dict)   : The dict with label Energy for each file.
             sep         (String) : Seperator for the file.
             
        Returns:
            A dataframe with columns as the optical constants.
    '''
    EnergyFile = pd.read_csv(filename,sep)
    df = EnergyFile.iloc[: , [labelEnergy["DeltaPara"],
                              labelEnergy["BetaPara"],
                              labelEnergy["DeltaPerp"],
                              labelEnergy["BetaPerp"],
                              labelEnergy["Energy"],
                              ]].copy() 
    df.columns=['DeltaPara','BetaPara','DeltaPerp','BetaPerp','Energy']
    df.sort_values(by=['Energy'],inplace=True)
    df.drop_duplicates(subset=['Energy'],keep=False,ignore_index=True,inplace=True)
    return df

In [ ]:
 def getInterpolatedValue(df,value):
    '''
    Returns the linearly interpolated value after removing the duplicates, if any
    
        Parameters:
            df  (DataFrame)      : A dataframe of energies created by generateDataFrame.
            value (double/float) : The energy value at which you want to interpolate.
        
        Returns:
            A list of interpolated optical properties for the given energy. 
    '''
    energy_id = 4
    nearest_id = df['Energy'].sub(280).abs().idxmin()
    numColumns = len(df.columns)
    valArray = np.zeros(numColumns);
    if(df.iloc[nearest_id][energy_id] > value):
        xp = [df.iloc[nearest_id - 1][energy_id], df.iloc[nearest_id ][energy_id]];
        for i in range(0,numColumns):
            yp = [df.iloc[nearest_id - 1][i], df.iloc[nearest_id][i]];
            valArray[i] = np.interp(value,xp,yp);

    elif (df.iloc[nearest_id][energy_id] < value):
        xp = [df.iloc[nearest_id][energy_id], df.iloc[nearest_id + 1][energy_id]];
        for i in range(0,numColumns):
            yp = [df.iloc[nearest_id][i], df.iloc[nearest_id + 1][i]];
            valArray[i] = np.interp(value,xp,yp);

    else:
        for i in range(0,numColumns):
            valArray[i] = df.iloc[nearest_id][i];
            
    return valArray[0:4].tolist();


In [ ]:
# Generating dataFrame for the text file.
filename='PEOlig2018.txt'
df=generateDataFrame(filename,labelEnergy)

# Step 1 : Providing Input Data Parameters

The Input Data for `Cy-RSoXS` has the following mandatory inputs:

- The set of energies you want to run.
- The physical dimensions in the order of (X,Y,Z). Note that HDF5 dimensions are in the order of (Z,Y,X)
- The PhysSize (in nm)
- The rotation Angle that you want to rotate the electric field

Failuare to provide any one of the input will flag an error and the code will not launch.

Additionally, there are optional input parameters for `Cy-RSoXS` as:
- The interpolation used : Nearest Neighbour or Trilinear interpolation (Default: Trilinear)
- Number of OpenMP threads: The minimum number of thread should be equal to number of GPU. (Deafult : 1)
- Windowing Type for FFT: Hanning or None


Key points:
-------------------------
- $Z$ axis corresponds to the thickness of the material
- $\vec{k} = (0,0,k)$  
- $\vec{E}$ field is rotated in $XY$ plane , $E_z = 0$

In [1]:
inputData = cy.InputData() # Create a object for Input Data

NameError: name 'cy' is not defined

In [ ]:
#Required dependecies:
energyList = [280.0,281.5,281.0]
inputData.setEnergies(energyList) 
inputData.physSize(5.0) # in nm
inputData.dimensions(X= 32,Y= 32,Z=16)
inputData.setRotationAngle(StartAngle = 0,EndAngle = 1,IncrementAngle = 5.0)

In [ ]:
#Optional dependencies
inputData.interpolationType = cy.InterpolationType.Linear
inputData.windowingType = cy.FFTWindowing.NoPadding

In [ ]:
inputData.validate() # Validate input Data. True means all required dependencies are present.

In [ ]:
inputData.print() # Check the input values

# Step 2 : Providing Refractive Index Constants 

The refractive index is passed in the form of `list` from python to Cy-RSoXS.
- The list is of the size (NumMaterial $\times$ 4)
- The list eneteries for each material must be in the order of [$\delta_{\parallel}$,$\beta_{\parallel}$, $\delta_{\perp}$, $\beta_{\perp}$]

In [ ]:
refractiveIndex = cy.RefractiveIndex(inputData)

In [ ]:
for energy in energyList:
    val = [getInterpolatedValue(df,energy),getInterpolatedValue(df,energy)]
    refractiveIndex.addData(OpticalConstants = val,Energy=energy)

In [ ]:
refractiveIndex.print() # Print the value to verify if its correct

In [ ]:
refractiveIndex.validate()

# Step 3 : Providing Voxel data

The Voxel data comprises of 2 component defined for each material :
- Aligned component   : A vector $(s_x,s_y,s_z)$ with alignment direction parallel to $z$ direction.
- Unaligned component : A scalar component

There are two ways of providing the voxelData:
- Directly from HDF5 file.
- From numpy arrays.

These approaches are mutually exclusive. They can not be combined


In [ ]:
voxelData = cy.VoxelData(inputData) #Create an object for Voxel Data

### Approach 1 : Through HDF5 file

It is straightforward. Just pass the HDF5 filename.

In [ ]:
voxelData.reset()
voxelData.readFromH5(Filename = 'edgespheres32.hd5')
# voxelData.writeToH5()

In [ ]:
voxelData.validate()

### Approach 2 :  Through numpy arrays

Make use of function `addVoxelData` to pass the numpy arrays. 

Remark 1: The code creates the copy of numpy arrays. If we want to pass it as a pointer, we would need to make sure that the memory layout of CyRSoXS is compatible with VoxelData in python. (Future work)

Remark 2: You are allowed to provide the entry to the material only one time. If you have provided multiple times. Then it will not add the entry and would return a WARNING. You can call `reset` to overcome this and add the entries from scratch.

In [ ]:
f = h5py.File('edgespheres32.hd5', 'r')
morph = f['vector_morphology']

In [ ]:
#Note to cast the input into the required shape. For single bit computation use np.single.
# Otherwise pybind will create a copy of memory while transferring the data to Cy-RSoXS
Mat_1_alignment = (morph['Mat_1_alignment'].value).astype(np.single)
Mat_2_alignment = (morph['Mat_2_alignment'].value).astype(np.single)
Mat_1_unaligned = (morph['Mat_1_unaligned'].value).astype(np.single)
Mat_2_unaligned = (morph['Mat_2_unaligned'].value).astype(np.single)

In [ ]:
f.close() # Close the file

In [ ]:
voxelData = cy.VoxelData(inputData)

In [ ]:
voxelData.reset() # Just to reset. 
#Note that you can specify only one way to allocate either through HDF5 file or numpy array. Not a combined way
voxelData.addVoxelData(Mat_1_alignment,Mat_1_unaligned,0)
voxelData.addVoxelData(Mat_2_alignment,Mat_2_unaligned,1)

# Step 4: Scattering Pattern 

- Allocate the memory to store scattering pattern. 


In [ ]:
scatteringPattern = cy.ScatteringPattern(inputData)

# Step 5: Lauch the GPU code

In [ ]:
%%time

with cy.ostream_redirect(stdout=True, stderr=True): # Redirects the std::cout output to Python console
        cy.launch(VoxelData = voxelData,RefractiveIndexData = refractiveIndex,InputData = inputData,ScatteringPattern=scatteringPattern)


# Step 6: Output

There are three ways to output the scattering pattern:

- Pass through numpy array
- Dump to HDF5 file 
- Dump to VTI file

In [ ]:
pattern = scatteringPattern.dataToNumpy(Energy=280) # returns Numpy array for that energy. Error if the energy is not found

In [ ]:
scatteringPattern.writeToHDF5() # Write to HDF5 all the energy currently simulated

In [ ]:
scatteringPattern.writeToVTI() # Write to VTI all the energy currently simulated

# Step 7: Cleanup

You can cleanup individually or all at once.

In [ ]:
#Clearing individually
voxelData.clear()
scatteringPattern.clear()
refractiveIndex.clear()

In [ ]:
cy.cleanup(VoxelData = voxelData,ScatteringPattern = scatteringPattern,RefractiveIndex = refractiveIndex)